# How to Build a Personalized Trading Dashboard? 
> A personalized dashboard to viusalize trading actions and stock development

- toc: false
- badges: true
- comments: true
- categories: [data analysis, trading, jupternotebook]

In [1]:
# hide 
from datetime import datetime, timedelta

import altair as alt
import numpy as np
import pandas as pd
import panel as pn
import yfinance as yf
from altair import datum
from utility import *

alt.renderers.enable("default")
pn.extension("vega")

copy_to_post = 1

TL;NR: if you are mainly interested in the codes, here is the link to [GitHub]()

# Motivation

Yesterday I came across an article [The Boredom Economy](https://www.nytimes.com/2021/02/20/business/gamestop-investing-economy.html). Sydney Ember explained the GameStop phenomenon as investors' reaction to the boredom experienced during the pandemic. 

Being one *amateur day traders* new to the market, I fully concur with the explanation Sydney put forth. I noticed myself spending hours daily *analyzing* the market and *trading* frequently for profits, as an escape from boredom. Resultantly, I I generated an amount of data in 2020 that could pertentially be used for a study on my 'trading style'. 

With this in mind, I decided started a project on analysing my trading activities. To start with, I create a dashboard to visualize the activities, to understand how and when I tend to buy/sell a stock. 

# Preparation

My primary trading platform is [Trading212](https://www.trading212.com). The platform recently included a [new feature](https://community.trading212.com/t/new-feature-export-your-investing-history/35612) that allows exporting transaction history in csv format. The exported data is clean neatly structured, which is ready for analysis. 

For the tools, I noticed a repo on GitHub [panel-altair-dashboard](https://github.com/bendoesdata/panel-altair-dashboard) that creates a simple, yet powerful, visualization tool (dashboard) in roughly 25 lines of codes. It is achieved by with Panel and Altair. 

To achieve visualizing my trading activities, I include the following features: 

- mark transaction actions (sell or buy), in the stock time history
- plot the stock historical data using candlestick



# Time to visualize the time series!

Below is a screenshot of the dashboard. In it, the transaction data (buy or sell)is visualzied along with the market data.  There is an option to visualize the market data in either line plot or candlestick, depending on whether you are interested in long term trends or the variations within the day. For a live demo, please refer to GitHub folders. 


It is still at the very early stage of development. In the future, I would like to add the following, 

- provide a summary of my portfolio 
- normalize the stock price for the selected during 

And more features will be inlcuded if I find anything interesting! 


![chart](./figs/trading_dashboard.png)

In [2]:
# hide
fln = "dummy_transactions.csv"
transactions = read_transactions(fln)

In [3]:
# hide
title = '# Trading Dashboard'
subtitle = 'A personalized visualization tool for **Trading 212** trading activities and market data'

companies = transactions["Ticker"].dropna().unique().tolist()
ticker = pn.widgets.Select(name="Company", options=companies)
style = pn.widgets.Select(name="Plot Style", options=["Candelstick", "Line"])

# this creates the date range slider
date_range_slider = pn.widgets.DateRangeSlider(
    name="Date Range",
    start=datetime(2020, 1, 1),
    end=datetime.today(),
    value=(datetime(2020, 1, 1), datetime.today()),
)

# tell Panel what your plot "depends" on.
@pn.depends(ticker.param.value, date_range_slider.param.value, style.param.value)
def get_plots(ticker, date_range, style):  # start function

    # filter based on ticker
    subset = transactions[transactions["Ticker"] == ticker]

    start_date = date_range_slider.value[
        0
    ]  # store the first date range slider value in a var
    end_date = date_range_slider.value[1]  # store the end date in a var

    ts = read_ticker_ts(ticker=ticker, start=start_date, end=end_date)

    if style == "Candelstick":
        chart = plot_transactions(subset, ts)
    else:
        chart = plot_transactions_2(subset, ts)

    return chart

dashboard = pn.Row(
    pn.Column(title, subtitle, ticker, style, date_range_slider),
    get_plots,  # draw chart function!
)

In [4]:
# hide
dashboard.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Select(name='Company', options=['DIS'], value='DIS')
        [3] Select(name='Plot Style', options=['Candelstick', 'Line'], value='Candelstick')
        [4] DateRangeSlider(end=datetime.datetime(2021, ..., name='Date Range', start=datetime.datetime(2020, ..., value=(datetime.datetime(2020, ..., value_throttled=(datetime.datetime(2020, ...)
    [1] ParamFunction(function)

In [5]:
# hide 
from shutil import copyfile

if copy_to_post:
    src = "transation_view_altair.ipynb"
    dst = "../blog_posts/_notebooks/2021-02-21-portfolio_analysis.ipynb"
    copyfile(src, dst)

    print("copied")

copied
